In [ ]:
adata_unintegrated = ""
integrated_adata = {}
tools = []
data_dir = ""
batch = ""
label = ""

Jupyter notebook for integration results

Author: Erno Hänninen

Title: integration_results.ipynb

Created: 2022-01-26

In [ ]:

from plottable import Table
from scib_metrics.benchmark import Benchmarker, BatchCorrection, BioConservation
import scanpy as sc
import scib
from scvi.model.utils import mde
import pymde
import shutil, os, sys
import warnings
sys.path.insert(0, "../../../Scripts") #Adding a path to be able to import the jupyter_functions
from jupyter_functions import *
warnings.filterwarnings('ignore')

### Prepare data, and run the integration metrics

In [ ]:
adata = sc.read(f"{data_dir}/unintegrated.h5ad")
for tool in tools:
    
    if not tool == "bbknn" and not tool == "seurat_cca" and not tool == "conos" and not tool == "seurat_rpca" and not tool == "fastmnn":
        adata_int = sc.read(integrated_adata[tool])
        adata.obsm[tool] = adata_int.obsm[tool]
    elif tool == "fastmnn":
        adata_int = scib.pp.read_seurat(integrated_adata[tool])
        adata.obsm[tool] = adata_int.obsm["FASTMNN"]
    elif not tool == "bbknn" and not tool == "conos":
        adata_int = scib.pp.read_seurat(integrated_adata[tool])        
        adata.obsm[tool] = adata_int.X.todense()
    
tools.append("unintegrated")

tools_to_benchmark = tools.copy()
#Remove the tools outputting a graph
if "bbknn" in tools_to_benchmark: tools_to_benchmark.remove("bbknn")
if "conos" in tools_to_benchmark: tools_to_benchmark.remove("conos")
print(tools)
print(tools_to_benchmark)

#adata.write(os.getcwd() + "/integrated_embed.h5ad")


batch_correction = BatchCorrection(
    ilisi_knn = False
)

bio_conservation = BioConservation(
    clisi_knn=False
)

bm = Benchmarker(
    adata,
    batch_key=batch,
    label_key=label,
    batch_correction_metrics=batch_correction,
    bio_conservation_metrics = bio_conservation,
    embedding_obsm_keys=tools_to_benchmark,
    n_jobs=12,
)
bm.benchmark()

In [ ]:
print(adata)

In [ ]:
output_dir = os.path.dirname(data_dir)
metrics_dir = output_dir + "/Metrics"
if os.path.exists(metrics_dir):
    shutil.rmtree(metrics_dir)
os.makedirs(metrics_dir + "/Scaled")
os.makedirs(metrics_dir + "/Non_scaled")

### Plot scaled integration metrics

In [ ]:
bm.plot_results_table(save_dir = metrics_dir + "/Scaled")

### Plot non-scaled integration merics

In [ ]:
bm.plot_results_table(min_max_scale=False, save_dir = metrics_dir + "/Non_scaled")

In [ ]:
from rich import print
df = bm.get_results(min_max_scale=False)
df.to_csv(metrics_dir + "/non_scaled_metrics.csv")


### Unintegrated

In [ ]:
if "unintegrated" in tools:
    adata_unintegrated = adata.copy()
    sc.pp.neighbors(adata_unintegrated, use_rep="X_pca")
    sc.tl.umap(adata_unintegrated)
    sc.pl.umap(adata_unintegrated,color=label)
    sc.pl.umap(adata_unintegrated,color=batch)

### SCVI

In [ ]:
if "scvi" in tools:
    adata_scvi = adata.copy()
    # use scVI latent space for UMAP generation
    sc.pp.neighbors(adata_scvi, use_rep="scvi")
    sc.tl.umap(adata_scvi)
    print(adata_scvi)
    
    sc.pl.umap(adata_scvi,color=label)
    sc.pl.umap(adata_scvi,color=batch)
    
    
    adata_scvi.obsm["X_mde"] = mde(adata_scvi.obsm["scvi"])
    print("pymde plot")
    sc.pl.embedding(adata_scvi,basis="X_mde",color=[label])
    sc.pl.embedding(adata_scvi,basis="X_mde",color=[batch])
    
    

### SCANVI

In [ ]:
if "scanvi" in tools:
    adata_scanvi = adata.copy()
    # use scVI latent space for UMAP generation
    sc.pp.neighbors(adata_scanvi, use_rep="scanvi")
    sc.tl.umap(adata_scanvi)
    print(adata_scanvi)
    
    sc.pl.umap(adata_scanvi,color=label)
    sc.pl.umap(adata_scanvi,color=batch)
    
    adata_scanvi.obsm["X_mde"] = mde(adata_scanvi.obsm["scanvi"])
    print("pymde plot")
    sc.pl.embedding(adata_scanvi,basis="X_mde",color=[label])
    sc.pl.embedding(adata_scanvi,basis="X_mde",color=[batch])
    

### SCGEN

In [ ]:
if "scgen" in tools:
    adata_scgen = adata.copy()
    sc.pp.neighbors(adata_scgen, use_rep="scgen")
    sc.tl.umap(adata_scgen)
    print(adata_scgen)

    sc.pl.umap(adata_scgen,color=label)
    sc.pl.umap(adata_scgen,color=batch)

### TRVAE

In [ ]:
if "trvae" in tools:
    adata_trvae = adata.copy()
    
    sc.pp.neighbors(adata_trvae, use_rep="trvae")
    sc.tl.umap(adata_trvae)
    print(adata_trvae)
 
    sc.pl.umap(adata_trvae,color=label)
    sc.pl.umap(adata_trvae,color=batch)

### Harmony

In [ ]:
if "harmony" in tools:
    adata_harmony = adata.copy()
    
    sc.pp.neighbors(adata_harmony, use_rep="harmony")
    sc.tl.umap(adata_harmony)
    print(adata_harmony)
 
    sc.pl.umap(adata_harmony,color=label)
    sc.pl.umap(adata_harmony,color=batch)

### Combat

In [ ]:
if "combat" in tools:
    adata_combat = adata.copy()
    
    sc.pp.neighbors(adata_combat, use_rep="combat")
    sc.tl.umap(adata_combat)
    print(adata_combat)
 
    sc.pl.umap(adata_combat,color=label)
    sc.pl.umap(adata_combat,color=batch)


### Scanorama

In [ ]:
if "scanorama" in tools:
    adata_scanorama = adata.copy()
    
    sc.pp.neighbors(adata_scanorama, use_rep="scanorama")
    sc.tl.umap(adata_scanorama)
    print(adata_scanorama)
 
    sc.pl.umap(adata_scanorama,color=label)
    sc.pl.umap(adata_scanorama,color=batch)

### Desc

In [ ]:
if "desc" in tools:
    adata_desc = adata.copy()
    
    sc.pp.neighbors(adata_desc, use_rep="desc")
    sc.tl.umap(adata_desc)
    print(adata_desc)
 
    sc.pl.umap(adata_desc,color=label)
    sc.pl.umap(adata_desc,color=batch)

### Liger

In [ ]:
if "liger" in tools:
    adata_liger = adata.copy()
    
    sc.pp.neighbors(adata_liger, use_rep="liger")
    sc.tl.umap(adata_liger)
    print(adata_liger)
 
    sc.pl.umap(adata_liger,color=label)
    sc.pl.umap(adata_liger,color=batch)

### SAUCIE

In [ ]:
if "saucie" in tools:
    adata_saucie = adata.copy()
    
    sc.pp.neighbors(adata_saucie, use_rep="saucie")
    sc.tl.umap(adata_saucie)
    print(adata_saucie)
 
    sc.pl.umap(adata_saucie,color=label)
    sc.pl.umap(adata_saucie,color=batch)

### Seurat cca

In [ ]:
if "seurat_cca" in tools:
    adata_cca = adata.copy()
    
    sc.pp.neighbors(adata_cca, use_rep="seurat_cca")
    sc.tl.umap(adata_cca)
    print(adata_cca)
 
    sc.pl.umap(adata_cca,color=label)
    sc.pl.umap(adata_cca,color=batch)

### Seurat rpca

In [ ]:
if "seurat_rpca" in tools:
    adata_rpca = adata.copy()
    
    sc.pp.neighbors(adata_rpca, use_rep="seurat_rpca")
    sc.tl.umap(adata_rpca)
    print(adata_rpca)
 
    sc.pl.umap(adata_rpca,color=label)
    sc.pl.umap(adata_rpca,color=batch)

### Fastmnn

In [ ]:
if "fastmnn" in tools:
    adata_fastmnn = adata.copy()
    
    sc.pp.neighbors(adata_fastmnn, use_rep="fastmnn")
    sc.tl.umap(adata_fastmnn)
    print(adata_fastmnn)
 
    sc.pl.umap(adata_fastmnn,color=label)
    sc.pl.umap(adata_fastmnn,color=batch)

### MNN

In [ ]:
if "mnn" in tools:
    adata_mnn = adata.copy()
    
    sc.pp.neighbors(adata_mnn, use_rep="mnn")
    sc.tl.umap(adata_adata_mnnrpca)
 
    sc.pl.umap(adata_mnn,color=label)
    sc.pl.umap(adata_mnn,color=batch)

### BBKNN

In [ ]:
if "bbknn" in tools:
    adata_bbknn = sc.read(integrated_adata["bbknn"])
    sc.tl.umap(adata_bbknn)
    print(adata_bbknn)
    sc.pl.umap(adata_bbknn,color=label)
    sc.pl.umap(adata_bbknn,color=batch)

### Conos

In [ ]:
if "conos2" in tools:
    adata_conos = read_conos_to_scanpy(integrated_adata["conos"], adata)
    print(adata_conos)
    sc.pl.umap(adata_conos, color=label)
    sc.pl.umap(adata_conos, color=batch)